In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [2]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [3]:
from utils.navbar import NavBarBball

In [4]:
navbarObject = NavBarBball()
navbar = navbarObject.construct_nav()

In [5]:
fig = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2])])

In [6]:
from utils.news import bballNews
newsObject = bballNews()
newsBox = newsObject.construct_news_pane()

In [7]:
from utils.tabs import BballTabs
tabObject = BballTabs()
tabs = BballTabs.constructTabs(tabObject,4)

In [8]:
import dash_bootstrap_components as dbc

external_stylesheets = [dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets,suppress_callback_exceptions=True)


In [9]:
# Define the index page layout
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    navbar, 
    html.Br(),
    html.Div(id='page-content', children=[]), 
])

In [10]:
from pages import homepage

In [11]:
from pages import playerSearch

In [12]:
from pages import singlePlayer

In [13]:
@app.callback(Output('tabs-content-inline-3', 'children'),
              Input('tabs-inline', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return dbc.Row([
                    dbc.Col(dcc.Graph(figure=fig),width=6),
                    dbc.Col(dcc.Graph(figure=fig),width=6)
                ])
    elif tab == 'tab-2':
        return html.Div([
            html.Img(src="https://cdn.vox-cdn.com/thumbor/O-tzXeJOjOWZpeoJP5JHRCajHFs=/1400x1400/filters:format(jpeg)/cdn.vox-cdn.com/uploads/chorus_asset/file/24432116/1247055249.jpg",
                                        style={
                                    'max-width': '100%',
                                    'height': 'auto',
                                })
        ])
    elif tab == 'tab-3':
        return newsBox
    elif tab == 'tab-4':
        return dcc.Markdown(children = '''
        ## Markdown
        Yay!
        ''')


In [14]:
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/players':
        return playerSearch.layout
    if pathname == '/singlePlayer':
        return singlePlayer.layout
    elif pathname == '/':
        return homepage.layout
    elif pathname == '/home':
        return homepage.layout
    else: # if redirected to unknown link
        return "404 Page Error! Please choose a link"


In [15]:
from utils.players.players import Player
player = Player(-1)
@app.callback(
    Output(component_id='player-output', component_property='children'),
    Input(component_id='player-search', component_property='value')
)
def construct_player_table(name):
    get_players = player.getAllPlayers()
    if name == '':
        filtered_players = get_players
    else:
        filtered_players = [player for player in get_players if name in player['Name']] 
    table_header = [
        html.Thead(html.Tr([html.Th("Name"), html.Th("Team"), html.Th("Info")]))
    ]
    
    rows = [html.Tr([html.Td(player["Name"]), 
                     html.Td(player["Team"]), 
                     html.Td(html.A("↗", href="/singlePlayer/" + str(player["id"])))]) for player in filtered_players]
    table_body = [html.Tbody(rows)]
    
    return dbc.Table(table_header + table_body, striped=True, bordered=True, hover=True)
    

In [16]:
if __name__ == '__main__':
    app.run_server(host="0.0.0.0", mode='external', debug=True, port=8051)

 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


Dash app running on http://0.0.0.0:8051/
